In [ ]:
import numpy
import matplotlib
from bs4 import BeautifulSoup
import requests
import os

url = "https://www.wg-gesucht.de/wg-zimmer-und-1-zimmer-wohnungen-und-wohnungen-und-haeuser-in-Kaiserslautern.67.0+1+2+3.1.0.html"
url1 = "https://www.wg-gesucht.de/wg-zimmer-und-1-zimmer-wohnungen-und-wohnungen-und-haeuser-in-Kaiserslautern.67.0+1+2+3.1.1.html"
url2 = "https://www.wg-gesucht.de/wg-zimmer-und-1-zimmer-wohnungen-und-wohnungen-und-haeuser-in-Kaiserslautern.67.0+1+2+3.1.2.html"
url3 = "https://www.wg-gesucht.de/wg-zimmer-und-1-zimmer-wohnungen-und-wohnungen-und-haeuser-in-Kaiserslautern.67.0+1+2+3.1.3.html"
url4 = "https://www.wg-gesucht.de/wg-zimmer-und-1-zimmer-wohnungen-und-wohnungen-und-haeuser-in-Kaiserslautern.67.0+1+2+3.1.4.html"
url5 = "https://www.wg-gesucht.de/wg-zimmer-und-1-zimmer-wohnungen-und-wohnungen-und-haeuser-in-Kaiserslautern.67.0+1+2+3.1.5.html"

url_list = [url,url1,url2,url3,url4,url5] # creating a url list <- this can partly be automated but for the sake of simplicity and security I stay static!!!

def get_inserates(url):
  req = requests.get(url)
  soup = BeautifulSoup(req.text, "lxml")
  inserates = soup.find_all("div",{"class":"col-sm-8 card_body"})
  inserates = inserates[1:] # delete first advertisement element
  return inserates

def find_values(inserates):
  a_list = [] # space
  b_list = [] # location
  c_list = [] # type
  d_list = [] # cost  
  for inserate in inserates:
    description = inserate.find("div",{"class":"col-xs-11"})
    bc_keywords = description.text.replace(" ", "").replace("\n", "").split("|")
    bc_keywords = bc_keywords[0:2]
    # find values in inserate html
    a = inserate.find("div",{"col-xs-3 text-right"}).text.replace("\n", "")[0:-3]
    b = bc_keywords[0]
    c = bc_keywords[1]
    d = inserate.find("div",{"col-xs-3"}).text.replace("\n", "")[0:-2]
    # append list
    a_list.append(a)
    b_list.append(b)
    c_list.append(c)
    d_list.append(d)
  return a_list,b_list,c_list,d_list

def get_full_values(url_list):
  a_full = [] # space
  b_full = [] # location
  c_full = [] # type
  d_full = [] # cost  
  for url in url_list:
    inserates = get_inserates(url)
    a,b,c,d = find_values(inserates)
    a_full = a_full + a
    b_full = b_full + b
    c_full = c_full + c
    d_full = d_full + d
  return a_full, b_full, c_full, d_full

def get_dataset_list(a_list, b_list, c_list, d_list):
  loc_dict = {}
  loc_num = 0
  type_dict = {}
  type_num = 0
  embedded = []
  # Creating Dataset from Values
  for (a,b,c,d) in zip(a_list, b_list, c_list, d_list):
    # get b value with dictionary
    if b in loc_dict.keys():
      b = loc_dict[b]
    else:
      loc_dict[b] = loc_num
      b = loc_num
      loc_num += 1

    # get c value with dictionary  
    if c in type_dict.keys():
      c = type_dict[c]
    else:
      type_dict[c] = type_num
      c = type_num
      type_num += 1 
    if d: # eliminate falsy samples
      embedded.append(";".join([str(a),str(b),str(c),str(d)]))

  return embedded

a_list, b_list, c_list, d_list = get_full_values(url_list)
sample_list = get_dataset_list(a_list, b_list, c_list, d_list)
with open('dataset.txt', 'w') as f:
    for item in sample_list:
        f.write("%s\n" % item)